# Bayesian change point model

In [ ]:
# import dependencies 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymc as pm
import arviz as az
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
  

## load the module

In [ ]:
from src.brent_change_point_analysis import (load_and_prepare_data,exploratory_analysis,bayesian_change_point_model,interpret_model)

## Load and prepare_data

In [ ]:
 # Load and prepare data
price_data, event_data = load_and_prepare_data()
print("Data loaded and prepared successfully.")
#to print the first few rows of the data
print("Price Data:")
print(price_data.head())
print("Event Data:")
print(event_data.head())

    


## Exploratory Data Analysis

In [ ]:
 # Perform EDA
exploratory_analysis(price_data, event_data)
    


## Bayesian change point model

In [ ]:
    # Run Bayesian change point model
try:
        trace, idx = bayesian_change_point_model(price_data)
        
        # Interpret results
        change_point_date, insights, mu_1_mean, mu_2_mean, price_change, associated_events = interpret_model(trace, price_data, event_data)
        
        # Print insights
        print("\nChange Point Insights:")
        for insight in insights:
            print(insight)
        
        # Plot price series with change point
        plt.figure(figsize=(12, 6))
        plt.plot(price_data.index, price_data['Brent_Price'], label='Brent Oil Price')
        plt.axvline(x=change_point_date, color='g', linestyle='--', label=f'Change Point: {change_point_date.date()}')
        for event_date in event_data['Start_Date']:
            plt.axvline(x=event_date, color='r', linestyle='--', alpha=0.5, label='Events' if event_date == event_data['Start_Date'].iloc[0] else '')
        plt.title('Brent Oil Prices with Detected Change Point')
        plt.xlabel('Date')
        plt.ylabel('Price (USD/bbl)')
        plt.legend()
        plt.savefig('brent_change_point.png')
        plt.show()
        
        # Save results
        results = pd.DataFrame({
            'Change_Point_Date': [change_point_date],
            'Mean_Before': [mu_1_mean],
            'Mean_After': [mu_2_mean],
            'Price_Change_Percent': [price_change],
            'Associated_Events': [', '.join(associated_events['Event_Name'].tolist()) if not associated_events.empty else 'None']
        })
        results.to_csv('change_point_results.csv', index=False)
        print("\nResults saved to 'change_point_results.csv'")
    
except Exception as e:
        print(f"Error in change point modeling: {e}")
        print("Note: The sample data is too short (10 days in 1987) for reliable change point detection. "
              "Consider providing a longer time series (e.g., from EIA) covering 2010–2025 to align with events.")
